In [85]:
from torch import nn
from torch.nn import functional as F
from torch import optim
from torchvision import datasets

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x: x

import torch
torch.cuda.get_device_name(torch.cuda.current_device())

'NVIDIA GeForce RTX 3060 Laptop GPU'

#### Load data

In [3]:
noisy_imgs_1, noisy_imgs_2 = torch.load('../data/train_data.pkl')

In [4]:
noisy_imgs_1.shape

torch.Size([50000, 3, 32, 32])

In [72]:
noisy_imgs_2.shape

torch.Size([50000, 3, 32, 32])

In [73]:
noisy_imgs, clean_imgs = torch.load('../data/val_data.pkl')

In [25]:
# def normalize(train_input,test_input):
#     mu, std = train_input.mean(), train_input.std()
#     train_input.sub_(mu).div_(std)
#     test_input.sub_(mu).div_(std)
#     return train_input,test_input

In [90]:
train_input,train_target=noisy_imgs_1.float()/255.0,noisy_imgs_2.float()/255.0
test_input,test_target=noisy_imgs.float()/255.0,clean_imgs.float()/255.0

In [91]:
model= nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 32)
    )
batch_size,nb_epochs=100,20
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)
mse=nn.MSELoss()

for e in range(nb_epochs):
    print('Epoch->',e,end=' ')
    losses=[]
    for b in range(0,train_input.size(0),batch_size):
        output = model(train_input.narrow(0, b, mini_batch_size))
        loss = mse(output, train_target.narrow(0, b, mini_batch_size))
        losses.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss=',sum(losses)/(train_input.size(0)/batch_size),end='\n')

Epoch-> 0 Loss= tensor(0.0258, grad_fn=<DivBackward0>)
Epoch-> 1 Loss= tensor(0.0161, grad_fn=<DivBackward0>)
Epoch-> 2 Loss= tensor(0.0159, grad_fn=<DivBackward0>)
Epoch-> 3 Loss= tensor(0.0159, grad_fn=<DivBackward0>)
Epoch-> 4 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 5 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 6 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 7 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 8 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 9 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 10 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 11 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 12 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 13 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 14 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 15 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 16 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Epoch-> 17 Loss= tensor(0.0158, grad_fn=<DivBackward0>)
Ep

In [98]:
def psnr(denoised,ground_truth):
    mse=torch.mean((denoised-ground_truth)**2)
    return  - 10 * torch.log10(((denoised-ground_truth) ** 2).mean((1,2,3))).mean()

In [100]:
model_outputs = []
for b in tqdm(range(0, test_input.size(0), batch_size)):
    output = model(test_input.narrow(0, b, batch_size))
    model_outputs.append(output)
model_outputs = torch.cat(model_outputs, dim=0)

output_psnr = psnr(model_outputs, test_target)
output_psnr

tensor(23.4655, grad_fn=<MulBackward0>)

In [ ]:
### For mini - project 1
class Model ():
    def __init__(self) -> None :
        ## instantiate model + optimizer + loss function + any other stuff you need

        pass

    def load_pretrained_model(self) -> None :
        ## This loads the parameters saved in bestmodel .pth into the model
        pass

    def train(self, train_input, train_target, num_epochs) -> None :
        #: train˙input : tensor of size (N, C, H, W) containing a noisy version of the images

        #: train˙target : tensor of size (N, C, H, W) containing another noisy version of the same images , which only differs from the input by their noise .
        pass

    def predict(self, test_input ) -> torch.Tensor:
        #:test_input : tensor of size (N1 , C, H, W) that has to be denoised by the trained or the loaded network .
        #: returns a tensor of the size (N1 , C, H, W)
        pass